In [1]:
from utils import get_config, join_sets, load_model, print_log, save_model, load_model_dict
from data import get_model_input
from model import get_best_model
from superdebug import debug
config = get_config("configs/debug_med_LM.yml", "_curation")
config["batch_size"] = 1
all_feature_columns, target, train_model_input, test_model_input, feature_names, original_feature_map, max_voted_users, train_data, test_data = get_model_input(config)
model, user_embedding = get_best_model(config, all_feature_columns, feature_names)
model = model.to(model.device)

为了使用邮件提醒功能，请设置环境变量MY_QQ_EMAIL（QQ邮箱地址）与MY_QQ_EMAIL_PWD（QQ邮箱授权码）
------------------ 2022-07-19 17:04:24 ------------------
GPU ready...
Smart using cuda:3
------------------ 2022-07-19 17:04:27 ------------------
DEBUG: at /home/TableSense/largedisk/wanrong/Curation-Modeling/data.py:188 get_model_input
Loading prepared data...
------------------ 2022-07-19 17:04:27 ------------------


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import numpy as np
from train import get_data_loader, pad_sequences
model = model.eval()
if "UPVOTED_USERS" in test_model_input:
    test_model_input["UPVOTED_USERS"] = pad_sequences(test_model_input["UPVOTED_USERS"], maxlen=max_voted_users, padding='post')
    test_model_input["DOWNVOTED_USERS"] = pad_sequences(test_model_input["DOWNVOTED_USERS"], maxlen=max_voted_users, padding='post')
test_model_input_list = [test_model_input[feature] for feature in model.feature_index if feature in test_model_input]

for i in range(len(test_model_input_list)):
    if len(test_model_input_list[i].shape) == 1:
        test_model_input_list[i] = np.expand_dims(test_model_input_list[i], axis=1)

tensor_data, test_loader = get_data_loader(model.lm_encoder is not None, test_model_input_list, test_model_input["input_ids"],test_model_input["token_type_ids"] if "token_type_ids" in test_model_input else None,test_model_input["attention_mask"], test_data[target].values, None, shuffle=False, batch_size=config["batch_size"])
test_input_list = []
for test_input in test_loader:
    test_input_list.append(test_input)

In [5]:
from matplotlib import interactive
import torch
from tqdm import tqdm
from train import convert_CTR_model_input, to_device, sample_updown_voted_users
y = test_data[target].values
print("Total user num:", user_embedding.shape[0])
for test_input in tqdm(test_input_list):
    CTR_model_input = convert_CTR_model_input(model, test_input, sample_voted_users=True, interactive = True)
    while CTR_model_input is not None:
        x, _y, weight = CTR_model_input
        print("User:", int(x[:,model.feature_index["USERNAME"][0]:model.feature_index["USERNAME"][1]]))
        y_pred = model(x).cpu().data.numpy()  # .squeeze()
        print(y_pred[0][0], "Upvote" if y_pred[0][0] > 0.5 else "Downvote")
        print("---------------------")
        CTR_model_input = convert_CTR_model_input(model, test_input, sample_voted_users=True, interactive = True)

Total user num: 346


  0%|          | 0/42643 [00:00<?, ?it/s]

Original upvoted users: []
New upvoted users: [1]
Original downvoted users: []
New downvoted users: []
User: 179
0.8025587 Upvote
---------------------
Original upvoted users: []
New upvoted users: [1, 2, 3, 4, 5, 6, 7, 8]
Original downvoted users: []
New downvoted users: []
User: 179
0.9964966 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: [1]
User: 179
0.77359784 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: [1, 2, 3, 4, 5, 6, 7, 8]
User: 179
0.94703543 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: []
User: 179
0.7471134 Upvote
---------------------
Original upvoted users: []
New upvoted users: [179]
Original downvoted users: []
New downvoted users: []
User: 179
0.81467545 Upvote
---------------------
Original upvoted users: []
New upvoted use

  0%|          | 1/42643 [07:27<5298:00:40, 447.28s/it]

Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: []
User: 310
0.8345919 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: []
User: 310
0.8345919 Upvote
---------------------
Original upvoted users: []
New upvoted users: [1]
Original downvoted users: []
New downvoted users: [2]
User: 310
0.86578417 Upvote
---------------------
Original upvoted users: []


  0%|          | 2/42643 [08:12<2494:12:58, 210.58s/it]

Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: [1]
User: 250
0.7646766 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: []
User: 250
0.7375101 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: [250]
User: 250
0.6722196 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: [250, 1, 2, 3, 4, 5, 6, 7]
User: 250
0.8875625 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: []
User: 250
0.7375101 Upvote
---------------------
Original upvoted users: []


  0%|          | 2/42643 [11:35<4120:45:58, 347.90s/it]


ValueError: invalid literal for int() with base 10: 'a'